In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')
import pandas as pd 
import numpy as np
df = pd.read_pickle('gdrive/My Drive/image_224x224_sale.pkl')
X = df['image_pixel']
y = df['high_sale']
X = X.to_numpy()
X = np.stack(X, axis=0)

Mounted at /content/gdrive


In [8]:
from sklearn.model_selection import train_test_split
# Split into 0.75:0.125:0.125
X = X/255.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.14286, random_state=42)
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of dev examples = " + str(X_dev.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(y_dev.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 1179
number of dev examples = 197
number of test examples = 197
X_train shape: (1179, 224, 224, 3)
Y_train shape: (1179,)
X_dev shape: (197, 224, 224, 3)
Y_dev shape: (197,)
X_test shape: (197, 224, 224, 3)
Y_test shape: (197,)


In [7]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet')
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.layers[0].trainable = False

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1000)              138357544 
_________________________________________________________________
flatten_2 (Flatten)          (None, 1000)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1001      
Total params: 138,358,545
Trainable params: 1,001
Non-trainable params: 138,357,544
_________________________________________________________________


In [ ]:
EPOCHS = 25
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_accuracy',
#     mode='max',
#     patience=6
# )

history = model.fit(
    X_train,
    y_train,
    epochs=20,
    validation_data=(X_dev, y_dev),
)

# history = model.fit_generator(
#     train_generator,
#     steps_per_epoch=50,
#     epochs=EPOCHS,
#     validation_data=validation_generator,
#     validation_steps=25,
#     callbacks=[early_stopping]
# )

Epoch 1/20
37/37 [==============================] - 569s 15s/step - loss: 0.6905 - accuracy: 0.5776 - val_loss: 0.6854 - val_accuracy: 0.6345
Epoch 2/20
37/37 [==============================] - 562s 15s/step - loss: 0.6874 - accuracy: 0.5827 - val_loss: 0.6811 - val_accuracy: 0.6345
Epoch 3/20
32/37 [========================>.....] - ETA: 1:03 - loss: 0.6845 - accuracy: 0.5879